In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"data\processed_df_for_consort.csv")

In [3]:
start = "first_contact_date"
end = 'therapy_starting_date'

In [6]:
df[df[start].isna()]

,clean_id,raw_id,clinic_application_date,intake_date,signing_date,suiteable_for_pp,First name,sheet,group,therapy_start_date,...,אי התאמה למחקר,"אין שת""פ טיפולי",משתתפים פעילים,נשירה מחקרית,נשירה קלינית- לאחר ת. טיפול,סיימו טיפול,עלייה לרמה 2,פספוסי גיוסים,first_contact_date,therapy_starting_date
23,a6048,A6048s,NaN,NaN,NaN,NaN,אוליביה,נשירה קלינית- לאחר ת. טיפול,Stepped Care,2025-02-28,...,0,0,0,0,1,0,0,0,NaN,2025-02-28
199,n4714,N4714s,NaN,NaN,NaN,1,נועם,משתתפים פעילים,NaN,NaN,...,0,0,1,0,0,0,0,0,NaN,NaN


In [7]:
df['waiting_duration'] =  pd.to_datetime(df[end]) - pd.to_datetime(df[start])
df['waiting_duration'] = df['waiting_duration'].dt.days

In [9]:
df[df[start].isna()]['waiting_duration'] > 670

23     False
199    False
Name: waiting_duration, dtype: bool

In [39]:
exclude = ['נשירה מחקרית',
 'נשירה קלינית- לאחר ת. טיפול',
 'אין שת"פ טיפולי',
 'סיימו טיפול',
 'אי הסכמה למחקר',
 'אי התאמה למחקר',
 'פספוסי גיוסים']

In [5]:
N = {
    'isin':['CAU', 'IPC-SSC', 'אי הסכמה למחקר', 'אי התאמה למחקר', 'אין שת"פ טיפולי', 'משתתפים פעילים', 'נשירה מחקרית', 'נשירה קלינית- לאחר ת. טיפול', 'סיימו טיפול', 'עלייה לרמה 2', 'פספוסי גיוסים'],
    'not_in': []
}



Eligible = {
            'isin':['CAU', 'IPC-SSC', 'אי הסכמה למחקר', 'אין שת"פ טיפולי', 'משתתפים פעילים', 'נשירה מחקרית', 'נשירה קלינית- לאחר ת. טיפול', 'סיימו טיפול', 'עלייה לרמה 2'],
            'not_in': []
}



Randomized = {
            'isin':['CAU', 'IPC-SSC', 'אין שת"פ טיפולי', 'משתתפים פעילים', 'נשירה מחקרית', 'נשירה קלינית- לאחר ת. טיפול', 'סיימו טיפול', 'עלייה לרמה 2'],
           'not_in': []
}


Dropout = {
           'isin':['נשירה מחקרית', 'נשירה קלינית- לאחר ת. טיפול'],
            'not_in': []
                    }

research_dropout = {
                    'isin':['נשירה מחקרית'],
                    'not_in': []
                    }

clinical_dropout = {
                    'isin':['נשירה קלינית- לאחר ת. טיפול'],
                    'not_in': []
                    }


in_waiting_list = {
    'isin': ['משתתפים פעילים'],
    'not_in': ['CAU', 'IPC-SSC'] + exclude
}


Finished = {
    'isin': ['סיימו טיפול'],
    'not_in': []
}


Active = {
    'isin': ['משתתפים פעילים', 'CAU', 'IPC-SSC'],
    'not_in': exclude
}

not_cooperative = {
    'isin': ['אין שת"פ טיפולי'],
    'not_in': []
}

In [6]:
consort_groups = {
    
    'N': N,
    'Eligible': Eligible,
    'Randomized': Randomized,
    'Dropout': Dropout,
    'Research Dropout': research_dropout,
    'Clinical Dropout': clinical_dropout,
    'In Waiting List': in_waiting_list,
    'Finished': Finished,
    'Active': Active,
    'Not Cooperative': not_cooperative

}

In [7]:


for key, val in consort_groups.items():
    for f in val:
        consort_groups[key][f] = [i for i in consort_groups[key][f] if i not in ['CAU', 'IPC-SSC']]

In [8]:
# Extracting the detials of how to calc the groups 

# rows = []
# for i,j in consort_groups.items():
#     row = {
#         "consort_group": i,
#         "included_tables": j["isin"],
#         "excluded_tables": j["not_in"]
#     }
#     rows.append(row)

# pd.DataFrame(rows).to_csv("consort_groups_definition.csv", index=0)

In [9]:
def isin_consort_group(row, consort_group):
    in_pos_sheet = False
    for sheet in consort_group['isin']:
        if row[sheet]:
            in_pos_sheet = True

    in_neg_sheet = False
    for sheet in consort_group['not_in']:
        if row[sheet]:
            in_neg_sheet = True

    return (in_pos_sheet and (not in_neg_sheet))

In [10]:
for consort_calc in consort_groups.keys():
    print(consort_calc)
    df[consort_calc] = df.apply(isin_consort_group, axis=1, args = [consort_groups[consort_calc]])

N
Eligible
Randomized
Dropout
Research Dropout
Clinical Dropout
In Waiting List
Finished
Active
Not Cooperative


In [11]:
for i in consort_groups.keys():
    print(i, df[i].sum())

N 310
Eligible 170
Randomized 134
Dropout 46
Research Dropout 30
Clinical Dropout 16
In Waiting List 37
Finished 31
Active 37
Not Cooperative 18


In [12]:
start = "first_contact_date"
end = 'therapy_starting_date'

In [13]:
df[start] = df[start].replace({'0': pd.NaT})
df[end] = df[end].replace({'0': pd.NaT})


In [14]:
df['waiting_duration'] =  pd.to_datetime(df[end]) - pd.to_datetime(df[start])
df['waiting_duration'] = df['waiting_duration'].dt.days

In [43]:
df['waiting_duration'].isna()

KeyError: 'waiting_duration'

In [22]:
# filtering out patients that've waited more than an year
df = df[~(df['waiting_duration'] > 670)]

In [17]:
df['did_started_therapy'] = pd.notna(df[end])

C:\Users\USER\AppData\Local\Temp\ipykernel_7672\1223032887.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['did_started_therapy'] = pd.notna(df[end])


In [29]:
df.waiting_duration.max()

593.0

In [30]:
df.shape

(309, 34)

In [28]:
df.group.unique()

array(['CAU', nan, 'Stepped Care', 'טיפול בקבוצת הורים '], dtype=object)

In [26]:
df[df.group == 'טיפול בקבוצת הורים']

,clean_id,raw_id,clinic_application_date,intake_date,signing_date,suiteable_for_pp,First name,sheet,group,therapy_start_date,...,Randomized,Dropout,Research Dropout,Clinical Dropout,In Waiting List,Finished,Active,Not Cooperative,waiting_duration,did_started_therapy


In [18]:
df['did_started_therapy']

0       True
1      False
2      False
3      False
4      False
       ...  
305     True
306    False
307     True
308    False
309    False
Name: did_started_therapy, Length: 309, dtype: bool

In [33]:
df.N.sum()

309

In [36]:
df["first_contact_date"].isna().sum()

2

In [19]:
# df.to_csv(r"data\export_with_days_delta.csv", index=0)
df.to_excel(r"data\export_with_days_delta.xlsx", index=0)

In [18]:
df.suiteable_for_pp == '1'

0      False
1       True
2       True
3       True
4       True
       ...  
305     True
306     True
307    False
308    False
309     True
Name: suiteable_for_pp, Length: 309, dtype: bool